In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
pd.set_option('display.max_columns', None)

### Load hospital base directory

In [2]:
df_hospitals = pd.read_json('../data/in/raw/atlas/locations.json', dtype={'zip': str})

### Web scrape departments of the hospitals

In [ ]:
# create new dataframe to store department infos for each hospital
df_departments = pd.DataFrame(df_hospitals['link'].apply(lambda x: x.split('/')[-2])).rename(columns={'link': 'hospital_id'})

In [4]:
def get_departments(hospital_id):

    # GET method
    request_url = f'https://bundes-klinik-atlas.de/krankenhaussuche/krankenhaus/{hospital_id}'
    response = requests.get(request_url)

    # parse HTML
    soup = BeautifulSoup(response.content, "html.parser")

    # extracting data
    result_list = soup.find(name='ul', class_='rte_ul')
    result_list_elements = result_list.find_all('li')

    # store departments and number of treatments in a list
    departments = {}
    for li in result_list_elements:
        try:
            department = str(li.text.split(':')[0])
            n_treatments = int(li.text.split(':')[1].strip().replace('.', ''))
            departments.update({f"{department}": n_treatments})
        except ValueError:
            department = str(li.text.split(':')[0])
            departments.update({f"{department.replace('\n', ' ').strip()}": 0})
    
    return departments

def bring_departments_to_df(df, departments, hospital_id):
    for department, n_treatments in departments.items():
        df.loc[df['hospital_id'] == hospital_id, department]= n_treatments
    return df

# get department info for each hospital and store it to dataframe
for hospital_id in df_departments.head(50)['hospital_id']:
    time.sleep(15)

    departments = get_departments(hospital_id)
    df_departments = bring_departments_to_df(df_departments, departments, hospital_id)

In [12]:
df[0:50].to_csv('../data/in/staging/atlas_0-49.csv', index=False, encoding='utf-8')

In [14]:
df_test = pd.read_csv('../data/in/staging/atlas_0-49.csv')
df_test.head(4)

,hospital_id,Innere Medizin,Allgemeine Chirurgie,Frauenheilkunde und Geburtshilfe,Geburtshilfe,Neonatologie,Intensivmedizin,Hämatologie und internistische Onkologie,Rheumatologie,Schmerztherapie,Pädiatrie,Innere Medizin/Schwerpunkt Kardiologie,Unfallchirurgie,Dermatologie,Innere Medizin/Schwerpunkt Geriatrie,Neurochirurgie,Geriatrie/Tagesklinik (für teilstationäre Pflegesätze),"Zahn- und Kieferheilkunde, Mund- und Kieferchirurgie",Neurologie,Innere Medizin/Schwerpunkt Gastroenterologie,Innere Medizin/Schwerpunkt Pneumologie,Strahlenheilkunde,Innere Medizin/Schwerpunkt Hämatologie und internistische Onkologie,Allgemeine Chirurgie/Schwerpunkt Gefäßchirurgie,Innere Medizin/Schwerpunkt Nephrologie,Pädiatrie/Schwerpunkt Neonatologie,Urologie,Geriatrie,"Hals-, Nasen-, Ohrenheilkunde",Palliativmedizin,Orthopädie und Unfallchirurgie,Sonstige Fachabteilung,Innere Medizin/ohne Differenzierung nach Schwerpunkten (II),Allgemeine Chirurgie/ohne Differenzierung nach Schwerpunkten (II),Geriatrie/ohne Differenzierung nach Schwerpunkten (II),Geriatrie/Nachtklinik (für teilstationäre Pflegesätze),Urologie/ohne Differenzierung nach Schwerpunkten (II),Kardiologie,Pneumologie,Nuklearmedizin,Sonstige Fachabteilung/ohne Differenzierung nach Schwerpunkten (II),Gefäßchirurgie,Herzchirurgie,Innere Medizin/Schwerpunkt Diabetes,Kinderkardiologie,Herzchirurgie/ Intensivmedizin (§ 13 Abs. 2 Satz 3 2. Halbsatz BPflV in der am 31.12.2003 geltenden Fassung),Allgemeine Chirurgie/Schwerpunkt Unfallchirurgie,Chirurgie/Schwerpunkt Orthopädie,Innere Medizin/Schwerpunkt Lungen- und Bronchialheilkunde,Allgemeine Chirurgie/Schwerpunkt Plastische Chirurgie,Orthopädie,Frauenheilkunde,Thoraxchirurgie,Wirbelsäulenchirurgie,Plastische Chirurgie,Augenheilkunde,Gastroenterologie,Endokrinologie,Innere Medizin/ohne Differenzierung nach Schwerpunkten (III),Hämatologie und internistische Onkologie/ohne Differenzierung nach Schwerpunkten (II),Operative Intensivmedizin/Schwerpunkt Chirurgie,"Neurologie/Schwerpunkt Schlaganfallpatienten (stroke units, Artikel 7 § 1 Abs. 3 GKV-SolG)",Neurologie/ohne Differenzierung nach Schwerpunkten (IV),Sonstige Fachabteilung/ohne Differenzierung nach Schwerpunkten (III),Intensivmedizin/Schwerpunkt Neurochirurgie,Sonstige Fachabteilung/ohne Differenzierung nach Schwerpunkten (IV),Dermatologie/ohne Differenzierung nach Schwerpunkten (III),Dermatologie/ohne Differenzierung nach Schwerpunkten (IV),Nephrologie,Augenheilkunde/ohne Differenzierung nach Schwerpunkten (II),Intensivmedizin/ohne Differenzierung nach Schwerpunkten (II),Hämatologie und internistische Onkologie/ohne Differenzierung nach Schwerpunkten (IV),Endokrinologie/ohne Differenzierung nach Schwerpunkten (III),Kinderchirurgie,Hämatologie und internistische Onkologie/Schwerpunkt Pädiatrie,Innere Medizin/Schwerpunkt Infektionskrankheiten,Kinderchirurgie/ohne Differenzierung nach Schwerpunkten (II),Angiologie,Allgemeine Chirurgie/ohne Differenzierung nach Schwerpunkten (IV)
0,771003,7598.0,6347.0,5825.0,1897.0,627.0,485.0,431.0,162.0,88.0,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,771011,2981.0,1761.0,521.0,NaN,NaN,227.0,NaN,NaN,NaN,NaN,4098.0,2230.0,958.0,449.0,187.0,126.0,21.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,771012,732.0,1128.0,2444.0,NaN,NaN,241.0,565.0,NaN,NaN,2242.0,1244.0,NaN,NaN,NaN,NaN,NaN,NaN,1910.0,1291.0,937.0,771.0,700.0,651.0,311.0,170.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [20]:
treatments_list = ['Chirurgischer Herzklappenersatz']
treatments_dict = {}
for treatment in treatments_list:
    treatments_dict[treatment] = treatment.replace(' ', '%20')

In [21]:
for hospital_id in df_test['hospital_id'].head(1):
    for treatment, treatment_searchlabel in treatments_dict.items():

        # GET method
        request_url = f'https://bundes-klinik-atlas.de/krankenhaussuche/krankenhaus/{hospital_id}/?tx_tverzhospitaldata_show%5Bsearchlabel%5D={treatment_searchlabel}'
        print(request_url)
        response = requests.get(request_url)

        # parse HTML
        soup = BeautifulSoup(response.content, "html.parser")

        # extracting data
        result_list = soup.find_all(name='div', class_='c-tacho-text__text')
        result_list_elements = result_list.find_all('li')



https://bundes-klinik-atlas.de/krankenhaussuche/krankenhaus/771003/?tx_tverzhospitaldata_show%5Bsearchlabel%5D=Chirurgischer%20Herzklappenersatz


In [53]:
treatment_count_number = int(result_list[0].contents[1].text.replace('.',''))
treatment_count_description = result_list[0].contents[2].text.replace('\n','').strip().replace('(','').replace(')','')

In [54]:
nursing_staff_quotient_count = result_list[1].contents[1].text
nursing_staff_quotient_description = result_list[1].contents[2].text.replace('\n','').strip().replace('(','').replace(')','')